In [58]:
from collections import Counter

import os
import shutil

import numpy as np
import pandas as pd

In [59]:
XLSX_PATH = '../data/Branch_2.xlsx'
INSTANCE_DIR = 'instance'

In [3]:
sheets = pd.read_excel(XLSX_PATH, sheet_name=[0, 1, 2])

In [4]:
providers, details, supplies = sheets[0], sheets[1], sheets[2]

## Ограничения целостности

### Поставщики 

#### SName NOT NULL
Отбрасывание записи

In [5]:
providers[providers['SName'].isna()].head()

,SID,SName,SCity,Address,Risk
240,240,NaN,Саратов,Учебная 21,3.0
900,900,NaN,﻿Москва,Южная 23,2.0


In [6]:
providers = providers.dropna(subset=['SName'])

In [7]:
providers[providers['SName'].isna()].head()

,SID,SName,SCity,Address,Risk


#### SCity NOT NULL
Замена пустого значения наиболее часто встречающимся значением города поставщика в рамках данного филиала

In [8]:
providers[providers['SCity'].isna()]

,SID,SName,SCity,Address,Risk
540,540,Феникс,NaN,Уральская 37,1.0
660,660,"ООО ""МТА""",NaN,Ереванская 46,2.0
780,780,"ООО ""Эльтадор-М""",NaN,Родькина 47,1.0
1020,1020,СлавКабель,NaN,Литовская 31,1.0


In [9]:
def apply_most_common_constraint(container, column_name, name_column=None, extra_f=None):
    container = container.dropna(subset=[column_name])
    def apply_each_row(row):
        row = row.copy()
        if pd.isna(row[column_name]) or (extra_f is not None and extra_f(row)):
            if name_column is not None:
                freq = Counter(container[container[name_column] == row[name_column]][column_name])
            else:
                freq = Counter(container[column_name])
            most_common = freq.most_common()[0][0]
            row[column_name] = most_common
        return row
    return apply_each_row

In [10]:
providers = providers.apply(
    apply_most_common_constraint(providers, 'SCity', 'SName'), 
    axis=1,
)

In [11]:
providers[providers['SCity'].isna()]

,SID,SName,SCity,Address,Risk


#### UNIQUE (SName, Address, SCity)
Отбрасывание записей-дубликатов 

In [12]:
providers_unique_subset = ['SName', 'Address', 'SCity']

In [13]:
providers[providers.duplicated(subset=providers_unique_subset, keep=False)].head()

,SID,SName,SCity,Address,Risk
179,179,ВАЛСИ,Новгород,Рылеева 6,2.0
180,180,ВАЛСИ,Новгород,Рылеева 6,3.0
599,599,ВАЛСИ,Ярославль,Южноуральская 32,2.0
600,600,ВАЛСИ,Ярославль,Южноуральская 32,3.0
839,839,Зет-Стоун,Воронеж,Украинская 19,1.0


In [14]:
providers = providers.drop_duplicates(subset=providers_unique_subset, keep='first')

In [15]:
providers[providers.duplicated(subset=providers_unique_subset, keep=False)].head()

,SID,SName,SCity,Address,Risk


#### Risk in (1, 2, 3)  
Замена ошибочного значения наиболее часто встречающимся значением риска сотрудничества в рамках данного города данного филиала

In [16]:
possible_risks = [1, 2, 3]

In [17]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))]

,SID,SName,SCity,Address,Risk
60,60,Мотэк-99,Уфа,Житомирская 12,NaN
120,120,Феникс,Ульяновск,Майкова 12,NaN
420,420,﻿Синапсис,Ульяновск,Львовская 22,NaN
480,480,Мотэк-99,Самара,Ростовская 49,NaN
960,960,Феникс,Уфа,Евтеева 14,NaN


In [18]:
invalid_risk_rows = np.logical_not(providers['Risk'].isin(possible_risks))
providers = providers.apply(
    apply_most_common_constraint(providers, 'Risk', 'SName', extra_f=lambda e: e['Risk'] not in possible_risks),
    axis=1,
)

In [19]:
providers[np.logical_not(providers['Risk'].isin(possible_risks))]

,SID,SName,SCity,Address,Risk


### Детали

#### PName NOT NULL
Отбрасывание записи

In [20]:
details[details['PName'].isna()]

,PID,PName,PCity,Color,Weight
240,240,NaN,Саратов,Зелёный,1.821
300,300,NaN,﻿Москва,Серый,1.410
720,720,NaN,Ульяновск,Серый,0.466
1020,1020,NaN,Ульяновск,Синий,1.646


In [21]:
details = details.dropna(subset=['PName'])

In [22]:
details[details['PName'].isna()]

,PID,PName,PCity,Color,Weight


#### PCity NOT NULL
Замена пустого значения наиболее часто встречающимся значением города детали в рамках данного филиала

In [23]:
details[details['PCity'].isna()].head()

,PID,PName,PCity,Color,Weight
0,0,﻿Гайка,NaN,Чёрный,0.852
60,60,Ручка,NaN,Жёлтый,1.398
600,600,Кожух,NaN,Синий,12.194
900,900,Отвёртка,NaN,Жёлтый,0.312
960,960,﻿Гайка,NaN,Красный,1.673


In [24]:
details = details.apply(
    apply_most_common_constraint(details, 'PCity', 'PName'),
    axis=1,
)

In [25]:
details[details['PCity'].isna()].head()

,PID,PName,PCity,Color,Weight


#### Weight > 0  
Замена ошибочного значения средним значением веса деталей в рамках данного города данного филиала

In [26]:
details[(details['Weight'] <= 0) | details['Weight'].isna()].head()

,PID,PName,PCity,Color,Weight
180,180,Отвёртка,Самара,Синий,-1.0
540,540,Ручка,Саратов,Красный,-1.0
660,660,Отвёртка,Уфа,Зелёный,-1.0
780,780,Ручка,Самара,Белый,-1.0
840,840,Кожух,Саратов,Чёрный,-1.0


In [27]:
mean_weight = details[details['Weight'] > 0]['Weight'].mean()

In [28]:
details[details['Weight'] <= 0] = mean_weight
details[details['Weight'].isna()] = mean_weight

In [29]:
details[(details['Weight'] <= 0) | details['Weight'].isna()].head()

,PID,PName,PCity,Color,Weight


#### UNIQUE (PName, PCity, Color)  
Отбрасывание записей-дубликатов

In [30]:
details_unique_subset = ['PName', 'PCity', 'Color']

In [31]:
details[details.duplicated(subset=details_unique_subset, keep=False)].head()

,PID,PName,PCity,Color,Weight
4,4.000000,Отвёртка,Новгород,Жёлтый,0.162000
119,119.000000,Радиатор,Барнаул,Чёрный,10.256000
120,120.000000,Радиатор,Барнаул,Чёрный,18.671000
180,7.272451,7.27245,7.27245,7.27245,7.272451
359,359.000000,Радиатор,Ростов-на-Дону,Зелёный,12.282000


In [32]:
details = details.drop_duplicates(subset=details_unique_subset, keep='first')

In [33]:
details[details.duplicated(subset=details_unique_subset, keep=False)].head()

,PID,PName,PCity,Color,Weight


### Поставки

#### ShipDate NOT NULL
Отбрасывание записи

In [34]:
supplies[supplies['ShipDate'].isna()]

,SPID,SID,PID,Quantity,Price,ShipDate
5400,5400,117,164,687,369.04,NaT
9000,9000,375,476,1,492.61,NaT
9600,9600,272,1149,53,470.73,NaT
11400,11400,100,533,659,493.28,NaT


In [35]:
supplies = supplies.dropna(subset=['ShipDate'])

In [36]:
supplies[supplies['ShipDate'].isna()]

,SPID,SID,PID,Quantity,Price,ShipDate


#### Qty > 0  
Замена ошибочного значения наиболее часто встречающимся значением риска сотрудничества в рамках данного города данного филиала

In [37]:
supplies[supplies['Quantity'] <= 0].head()

,SPID,SID,PID,Quantity,Price,ShipDate
1200,1200,219,26,-1,72.33,2014-05-05
1800,1800,1020,816,-1,381.99,2013-06-14
3000,3000,990,431,-1,298.71,2013-03-20
4800,4800,144,776,-1,393.41,2014-05-17
6000,6000,205,573,-1,219.51,2014-12-11


In [38]:
supplies = supplies.apply(
    apply_most_common_constraint(supplies, 'Quantity', extra_f=lambda e: e['Quantity'] <= 0), 
    axis=1,
)

In [39]:
supplies[supplies['Quantity'] <= 0].head()

,SPID,SID,PID,Quantity,Price,ShipDate


#### Price > 0
Замена ошибочного значения средним значением цены деталей в рамках данного города данного филиала

In [40]:
supplies['Price'] = pd.to_numeric(supplies['Price'], errors='coerce')

In [41]:
supplies[supplies['Price'].isna() | (supplies['Price'] <= 0)].head()

,SPID,SID,PID,Quantity,Price,ShipDate
1,1,714,603,35,NaN,2014-04-23
184,184,371,1150,5,NaN,2013-12-05
201,201,152,775,8,NaN,2014-08-22
578,578,61,886,56,NaN,2013-09-17
654,654,355,137,454,NaN,2014-11-04


In [42]:
mean_price = supplies[np.logical_not(supplies['Price'].isna() | (supplies['Price'] <= 0))]['Price'].mean()

In [43]:
supplies[supplies['Price'].isna() | (supplies['Price'] <= 0)] = mean_price

In [44]:
supplies[supplies['Price'].isna() | (supplies['Price'] <= 0)].head()

,SPID,SID,PID,Quantity,Price,ShipDate


In [45]:
#### Уберем данные, которых уже нет

In [46]:
def apply_existance_constaints(row):
    row = row.copy()
    if (
        (row['PID'] not in details['PID']) 
        or (row['SID'] not in providers['SID'])
    ):
        row['SPID'] = None
    return row

In [47]:
supplies = supplies.apply(apply_existance_constaints, axis=1).dropna(subset=['SPID'])

#### SP.Qty * P.Weight <= 1500  
Отбрасывание записи

In [48]:
details[['PID', 'Weight']].head()

,PID,Weight
0,0.0,0.852
1,1.0,1.731
2,2.0,0.866
3,3.0,1.166
4,4.0,0.162


In [49]:
qty_and_weight = supplies[['PID', 'Quantity']].join(details[['PID', 'Weight']].set_index('PID'), on='PID', how='left')

In [50]:
qty_and_weight.head()

,PID,Quantity,Weight
0,303.0,144.0,13.833
2,648.0,23.0,19.232
3,265.0,22.0,1.586
4,114.0,250.0,0.945
5,1005.0,37.0,19.466


In [52]:
valid_supplies_indices = qty_and_weight['Quantity'] * qty_and_weight['Weight'] <= 1500

In [53]:
supplies = supplies[valid_supplies_indices]

## Сохранить результаты

In [69]:
shutil.rmtree(INSTANCE_DIR, ignore_errors=True)

In [70]:
os.mkdir('instance')

In [67]:
providers.to_csv('instance/s.csv', index=False)

AttributeError: module 'pandas' has no attribute 'to_csv'